# Welcome to Lumigator Foxfooding from [Mozilla.ai](https://www.mozilla.ai/)! 🐊 🦊

## Agenda

+ Working with Jupyter Notebooks
+ Lumigator Platform Overview  🐊
+ Understanding Machine Learning Workflows 
+ Thunderbird Dataset Walkthrough
+ Explanation of and Examination of Thunderbird Ground Truth
+ Model Selection ( 1 encoder/decoder), (2 decoder), eval against GPT4
+ Run experiment and show results
+ Evaluate results and discuss

## Jupyter Walkthrough

[Jupyter Notebooks](https://jupyter-notebook.readthedocs.io/en/stable/) are an executable code/text environment for (mostly) Python code. Our Jupyter environment is on JupyterHub, which you've launched at startup. To work with Jupyter, click "run cell" to run the code and see results below the cell you're currently running. Cells are executed sequentially 

## Vocabulary

Some terms you'll hear us using throughout the session: 

+ **LLM** - Large language model, a text-based model that performs next-word predictions 
+ **Tokens** - Words broken up into pieces to be used in an LLM 
+ **Inference** - The process of getting a prediction from a large language model 
+ **Embeddings** - Numerical representations of text generated by modeling
+ **Encoder-decoder models** - a neural network architecture comprised of two neural networks, an encoder that takes the input vectors from our data and creates an embedding of a fixed length, and a decoder, also a neural network, which takes the embeddings encoded as input and generates a static set of outputs such as translated text or a text summary
+ **Decoder-only models** - Receive a prompt of text directly and predict the next word
+ **Task** - Machine learning tasks to fit a specific model type, including translation, summarization, completion, etc. 


## Lumigator Platform Overview

TODO: Add URL

You'll be working in the Jupyter notebook and the platform itself is accessible via URL in the slide deck. The app itself consists of an API, which you can access and test out methods with in the [OpenAPI spec](https://swagger.io/specification/), at the platform URL, under docs. 

<img src="openapi.png" alt="drawing" width="200"/>

# Machine Learning Workflows

Generally, when working with machine learning, we are looking to generate a model artifact from data. We have several stages we care about: the data preprocessing, model training, model generation, inference, and evaluation. 

<img src="ml_workflow.png" alt="drawing" width="400"/>

Within the universe of modeling approaches, there are supervised and unsupervised approaches, as well as reinforcement learning. When we think of language modeling, that falls in the realm of neural network approaches. 


<img src="ml_family.png" alt="drawing" width="400"/>


Lumigator focuses on **inference** and **evaluation** for large language models: we want to be able to take our own dataset, perform inference on it, and evaluate the results to see if the model we would like to use produces good results for our use-cases. Use-cases include cases that are specific to our business. 


## Machine learning is alchemy!

When we think of traditional application workflows, we think of an example such as adding a button. We can clearly test that we've added a blue button to our application, and that it works correctly. Machine learning is not like this! It involves a lot of experimentation, tweaking of hyperparameters and prompts and trying different models. Expect for the process to be imperfect, with many iterative loops. Luckily, Lumigator helps take away the uncertainty of at least model selection :)

> There’s a self-congratulatory feeling in the air. We say things like “machine learning is the new electricity”. I’d like to offer an alternative metaphor: machine learning has become alchemy. - [Ben Recht and Ali Rahmi](https://archives.argmin.net/2017/12/05/kitchen-sinks/)


Ultimately, the final conclusion of whether a model is good is if humans think it's good. 

With that in mind, let's dive into setting up experiments with Lumigator to test our models!

In [ ]:
import time
import lumigator_demo as ld
import pandas as pd
import matplotlib.pyplot as plt
import os

from datasets import load_dataset
from IPython.display import clear_output

# wrap columns for inspection
pd.set_option('display.max_colwidth', 0)
# stylesheet for visibility
plt.style.use("fast")

# Experiments
We're grouping experiments by team name to organize the data, pick a team name below and run the cell. 

In [ ]:
# suggestion: "lumigator_enthusiasts", "your team name etc"
team_name = TEAM_NAME_HERE

In [ ]:
# 

What are LLMS? what are encoder-decoder, decoder models, working with text and tokens 

## Ground Truth for Models
[Generating GT](https://thunderbird.topicbox.com/groups/addons/T18e84db141355abd-M4cca8e3f9e4fee9ae14b9dbb/self-hosted-version-of-extension-is-incorrectly-appearing-in-atn)

## Generating Data for Ground Truth Evaluation

In order to generate a ground truth summary for our data, we first need an input dataset. In this case we use threads from the [Thunderbird public mailing list.](https://thunderbird.topicbox.com/latest).  In order to generate the ground truth and then later evaluate the model, we need at least 100 samples to start with, where a sample is a single email or single email conversation.

Our selection criteria: 

+ Collect 100 samples of email thread conversations, as recent as possible and fairly complete so they can be evaluated
+ Clean them of email formatting such as `>`
+ One consideration here will be that BART, the baseline model we're using, accepts 1024 token context window as input, i.e.  we have to have input email threads that are ~ approximately 1000 words, so keeping on the conservative side

Once we've collected them, we'd like to take a look at the data before we generate summaries. 

In [ ]:
# show information about the Thunderbird dataset
dataset_id = "db7ff8c2-a255-4d75-915d-77ba73affc53"
r = ld.dataset_info(dataset_id)

In [ ]:
# download the dataset into a pandas dataframe
df = ld.dataset_download(dataset_id)
df.head()

In [ ]:
import re
from string import punctuation

def preprocess_text(text:str):
    text = text.lower()  # Lowercase text
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    text = re.sub(r"\b[0-9]+\b\s*", "", text)
    return text

df["examples"].map(preprocess_text)

In [ ]:
# Examine a single sample 
# we define the data with examples
df['examples'].iloc[0]

In [ ]:
# Add a function to do some simple character counts for model input
df['char_count'] = df['examples'].str.len()

In [ ]:
df.head

In [ ]:
# Show statistics about characters count
df['char_count'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.hist(df['char_count'], bins=30)
ax.set_xlabel('Character Count')
ax.set_ylabel('Frequency')

stats = df['char_count'].describe().apply(lambda x: f"{x:.0f}")

# Add text boxes for statistics
plt.text(1.05, 0.95, stats.to_string(), 
         transform=ax.transAxes, verticalalignment='top')

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(right=0.75)

plt.show()

In [ ]:
#TODO

what is ground truth? how do we vibe-compare it between models? 

In [ ]:
## Perform Ground Truth Generation with Mistral 

responses = []

for sample in df['examples'][0:100]:
    response = ld.get_mistral_ground_truth(sample)
    print(f"Response from Mistral", response[:10])
    responses.append((sample, response))

In [ ]:
mistral_results_df = pd.DataFrame(responses, columns=['Original', 'Response'])

mistral_results_df

In [ ]:
# TODO

ways to run LLMs: API access, running on cluster, and running locally 
what is Ray? 

In [ ]:
# TODO

Lumigator and LM-Buddy, how they work together

In [ ]:
# Get all deployments 
ld.get_deployments()

In [ ]:
## Perform Ground Truth Generation with BART

# PUT DEPLOYMENT ID HERE 
deployment_id = "3b3a7f4d-0a4f-4ff1-b760-2c126279bc00"

for string in df['examples'][0:1]:
    response = ld.get_bart_ground_truth(deployment_id,"hello")
    print(string, response)
    responses.append((string, response))


## Loading Data

### Loading data
The following dataset is already in the format that we need as input: 
- one field called `examples` containing the text to summarize
- one field called `ground_truth` containing the summaries to the models' outputs against

Note that you can load many different types of file formats in a similar way (see https://huggingface.co/docs/datasets/loading#local-and-remote-files)

In [ ]:
#TODO:

huggingface datasets versus csv
and lm-buddy prefixes

## Dataset Upload

In [ ]:
dataset_name = "thunderbird.csv"
dataset_id = "f5d54efa-247d-4910-9393-f6003da9fb68" # thunderbird pre-saved dataset HuggingFace

r = ld.dataset_info(dataset_id)

## Model Selection

What you see below are different lists of models we have already tested for the summarization task.
The `models` variable at the end provides you with a selection, but you can choose any combination of them.

In [ ]:
enc_dec_models = [
    'hf://facebook/bart-large-cnn',
    'hf://mikeadimech/longformer-qmsum-meeting-summarization', 
    'hf://mrm8488/t5-base-finetuned-summarize-news',
    'hf://Falconsai/text_summarization',
]

dec_models = [
    'mistral://open-mistral-7b',
]

gpts = [
    "oai://gpt-4o-mini",
    "oai://gpt-4-turbo",
    "oai://gpt-3.5-turbo-0125"  
]

# TODO: add llamafile

models = [
    enc_dec_models[0], # bart-large-cnn
    dec_models[0], # Mistral-7B-Instruct-v0.3
    gpts[0] # gpt-4o-mini
]

# show selected models

models

In [ ]:
# TODO
Introduce metrics 

## Run Evaluations

In [ ]:
# change the following to 0 to use all samples in the dataset
max_samples = 10

responses = []
for model in models:
    descr = f"Testing {model} summarization model on {dataset_name}"
    responses.append(ld.experiments_submit(model, team_name, descr, dataset_id, max_samples))

In [ ]:
# TODO
Discuss Ray dashboard/show dashboard

### Track evaluation jobs

Run the following to track your evaluation jobs.

- *NOTE*: you won't be able to run other cells while this one is running. However, you can interrupt it whenever you want by clicking on the "stop" button above and run it at a later time.

In [ ]:
job_ids = [ld.get_resource_id(r) for r in responses]

wip = ld.show_experiment_statuses(job_ids)
while wip == True:
    time.sleep(5)
    clear_output()
    wip=ld.show_experiment_statuses(job_ids)

## Show evaluation results

In [ ]:
# after the jobs complete, gather evaluation results
eval_results = []
for job_id in job_ids:
    eval_results.append(ld.experiments_result_download(job_id))

# convert results into a pandas dataframe
eval_table = ld.eval_results_to_table(models, eval_results)

In [ ]:
eval_table

In [ ]:
eval_results[0]

## Evaluation Results

In [ ]:
#TODO

add eval discussion 